In [1]:
# ============================================================
# 00_download_esios.ipynb
# Descarga robusta del indicador ESIOS 600 (Precio mercado SPOT Diario)
# Periodo: 2024 completo (UTC) -> guarda CSV en data/raw/
# ============================================================

# -------- 0) Montar Google Drive --------
from google.colab import drive
drive.mount('/content/drive')

# -------- 1) Imports y dependencias --------
!pip -q install requests pandas python-dateutil python-dotenv

import os
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

# -------- 2) Configuración de rutas del proyecto --------
BASE_PATH = "/content/drive/MyDrive/energy_storage_esios"  # ajusta si tu carpeta se llama distinto
assert os.path.exists(BASE_PATH), f"No existe BASE_PATH: {BASE_PATH}. Revisa el nombre o ubicación en MyDrive."

RAW_DIR = os.path.join(BASE_PATH, "data", "raw")
SECRETS_DIR = os.path.join(BASE_PATH, "secrets")
ENV_PATH = os.path.join(SECRETS_DIR, ".env")

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(SECRETS_DIR, exist_ok=True)

print("BASE_PATH:", BASE_PATH)
print("RAW_DIR:", RAW_DIR)
print("SECRETS_DIR:", SECRETS_DIR)
print("ENV_PATH:", ENV_PATH)
print("Contenido BASE:", os.listdir(BASE_PATH))

# -------- 3) Cargar token desde secrets/.env --------
# El archivo debe contener una línea: ESIOS_TOKEN=xxxxxxxx
assert os.path.exists(ENV_PATH), (
    f"No existe el archivo .env en {ENV_PATH}. "
    "Créalo en Drive en la carpeta secrets con: ESIOS_TOKEN=TU_TOKEN"
)

load_dotenv(dotenv_path=ENV_PATH, override=True)
ESIOS_TOKEN = os.getenv("ESIOS_TOKEN")

assert ESIOS_TOKEN and len(ESIOS_TOKEN) > 10, (
    "No se ha cargado ESIOS_TOKEN o es demasiado corto. "
    "Revisa el contenido de secrets/.env (formato: ESIOS_TOKEN=TU_TOKEN)."
)

print("Token cargado correctamente | longitud:", len(ESIOS_TOKEN))

# -------- 4) Cliente API ESIOS --------
BASE_URL = "https://api.esios.ree.es"

def esios_headers(token: str) -> dict:
    # API v2: x-api-key es lo habitual. Mantengo Authorization por compatibilidad.
    return {
        "Accept": "application/json; application/vnd.esios-api-v2+json",
        "Content-Type": "application/json",
        "x-api-key": token,
        "Authorization": f"Token token={token}",
    }

def fetch_indicator(indicator_id: int, start_iso: str, end_iso: str, token: str) -> dict:
    url = f"{BASE_URL}/indicators/{indicator_id}"
    params = {"start_date": start_iso, "end_date": end_iso, "locale": "es"}
    r = requests.get(url, headers=esios_headers(token), params=params, timeout=90)
    r.raise_for_status()
    return r.json()

def payload_to_df(payload: dict) -> pd.DataFrame:
    values = payload.get("indicator", {}).get("values", [])
    df = pd.DataFrame(values)
    if df.empty:
        return df

    # Columnas típicas: datetime, value, geo_id, geo_name, etc.
    if "datetime" in df.columns:
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True, errors="coerce")

    if "value" in df.columns:
        df = df.rename(columns={"value": "price_eur_mwh"})

    # Nos quedamos con lo mínimo necesario, pero sin romper si faltan columnas
    keep_cols = [c for c in ["datetime", "price_eur_mwh", "geo_id", "geo_name"] if c in df.columns]
    df = df[keep_cols].copy()

    return df

# -------- 5) Descarga por meses (robusto) --------
INDICATOR_ID = 600  # Precio mercado SPOT Diario
start = datetime(2024, 1, 1)
end   = datetime(2025, 1, 1)

dfs = []
cur = start

while cur < end:
    nxt = min(cur + relativedelta(months=1), end)

    start_iso = cur.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_iso   = nxt.strftime("%Y-%m-%dT%H:%M:%SZ")

    print(f"\nDescargando {start_iso} -> {end_iso}")

    try:
        payload = fetch_indicator(INDICATOR_ID, start_iso, end_iso, ESIOS_TOKEN)
        dfm = payload_to_df(payload)
        print("  Registros:", len(dfm))

        if not dfm.empty:
            dfs.append(dfm)

    except requests.HTTPError as e:
        # Si hay error, mostramos el texto para diagnóstico
        print("  ERROR HTTP:", e)
        if hasattr(e, "response") and e.response is not None:
            print("  Status:", e.response.status_code)
            print("  Respuesta (primeros 500 chars):", e.response.text[:500])
        raise

    cur = nxt

assert dfs, "No se han descargado datos (lista dfs vacía). Revisa token/indicador/fechas."

df = pd.concat(dfs, ignore_index=True)

# -------- 6) Limpieza mínima: deduplicar, ordenar, sanity checks --------
df = df.drop_duplicates()
df = df.dropna(subset=["datetime", "price_eur_mwh"])
df = df.sort_values("datetime").reset_index(drop=True)

print("\nResumen:")
print("Shape:", df.shape)
print("Rango:", df["datetime"].min(), "->", df["datetime"].max())
print("Nulos:", df[["datetime", "price_eur_mwh"]].isna().sum().to_dict())

# -------- 7) Guardado a CSV --------
out_path = os.path.join(RAW_DIR, "esios_price_spot_2024.csv")
df.to_csv(out_path, index=False)
print("\nGuardado en:", out_path)

# -------- 8) Validación rápida de frecuencia (deltas) --------
deltas = df["datetime"].diff().value_counts().head(10)
print("\nDeltas más comunes entre registros:")
print(deltas)

# Muestra rápida
df.head()


Mounted at /content/drive
BASE_PATH: /content/drive/MyDrive/energy_storage_esios
RAW_DIR: /content/drive/MyDrive/energy_storage_esios/data/raw
SECRETS_DIR: /content/drive/MyDrive/energy_storage_esios/secrets
ENV_PATH: /content/drive/MyDrive/energy_storage_esios/secrets/.env
Contenido BASE: ['notebooks', 'data', 'results', 'src', 'secrets']
Token cargado correctamente | longitud: 64

Descargando 2024-01-01T00:00:00Z -> 2024-02-01T00:00:00Z
  Registros: 4470

Descargando 2024-02-01T00:00:00Z -> 2024-03-01T00:00:00Z
  Registros: 4110

Descargando 2024-03-01T00:00:00Z -> 2024-04-01T00:00:00Z
  Registros: 4470

Descargando 2024-04-01T00:00:00Z -> 2024-05-01T00:00:00Z
  Registros: 4326

Descargando 2024-05-01T00:00:00Z -> 2024-06-01T00:00:00Z
  Registros: 4470

Descargando 2024-06-01T00:00:00Z -> 2024-07-01T00:00:00Z
  Registros: 4326

Descargando 2024-07-01T00:00:00Z -> 2024-08-01T00:00:00Z
  Registros: 4446

Descargando 2024-08-01T00:00:00Z -> 2024-09-01T00:00:00Z
  Registros: 4470

Descar

,datetime,price_eur_mwh,geo_id,geo_name
0,2024-01-01 00:00:00+00:00,50.09,1,Portugal
1,2024-01-01 00:00:00+00:00,0.01,2,Francia
2,2024-01-01 00:00:00+00:00,50.09,3,España
3,2024-01-01 00:00:00+00:00,0.01,8826,Alemania
4,2024-01-01 00:00:00+00:00,0.01,8827,Bélgica
